# Recomender system using Spark

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()



<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

La bibliothèque Spark MLlib pour Machine Learning fournit une implémentation de filtrage collaboratif à l'aide des moindres carrés alternés (Alternating Least Squares). L'implémentation dans MLlib a ces paramètres:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.


In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



Nous pouvons faire une séparation pour évaluer les performances de notre modèle, mais gardez à l'esprit qu'il est très difficile de savoir de manière concluante à quel point un système de recommandation fonctionne vraiment pour certains sujets. Surtout si la subjectivité est impliquée, par exemple, tout le monde qui aime Star Wars ne va pas aimer le Star Trek, même si un système de recommandation peut suggérer le contraire.

In [7]:
(training, test) = data.randomSplit([0.8, 0.2])

In [8]:
# Construire une recommendation en utilisat ALS
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [9]:
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    19|  1.3275465|
|     31|   3.0|     7|  2.1409166|
|     31|   1.0|    29|-0.18804112|
|     85|   2.0|    20|  0.5647411|
|     65|   2.0|     5|   2.224414|
|     65|   2.0|    15|  0.9352038|
|     65|   1.0|     4|0.023433968|
|     53|   1.0|     6|  1.7077291|
|     53|   1.0|    23|  3.8076577|
|     53|   1.0|     7| 0.92047584|
|     78|   1.0|     1| 0.60433036|
|     78|   1.0|     4|  0.9783422|
|     78|   1.0|    11|  1.0314637|
|     34|   1.0|    28|  1.2500005|
|     34|   1.0|    16|  0.2469647|
|     34|   1.0|    19|  1.0586464|
|     34|   1.0|    15|  2.0560546|
|     34|   1.0|    17|   1.180474|
|     34|   3.0|    25|  1.1694434|
|     34|   1.0|    14| -0.8034165|
+-------+------+------+-----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.9715582471060988


Alors maintenant que nous avons le modèle, comment pourriez-vous réellement fournir une recommandation à un utilisateur?

De la même manière que nous l'avons fait avec les données de test! Par exemple:

In [12]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [13]:
# l'utilisateur 11 à 10 ratings dans les données de test 
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     12|    11|
|     18|    11|
|     19|    11|
|     22|    11|
|     30|    11|
|     39|    11|
|     40|    11|
|     51|    11|
|     67|    11|
|     71|    11|
|     72|    11|
|     77|    11|
|     78|    11|
|     82|    11|
|     89|    11|
+-------+------+



In [14]:
reccomendations = model.transform(single_user)

In [15]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     22|    11|  3.9857097|
|     39|    11|  3.9014015|
|     18|    11|  3.3222647|
|     30|    11|   2.490342|
|     51|    11|  1.9938318|
|     12|    11|  1.8084567|
|     19|    11|  1.6983118|
|     77|    11|  1.5228118|
|     78|    11|  1.0314637|
|     82|    11|  0.9510155|
|     71|    11| 0.32412025|
|     89|    11| 0.14838052|
|     40|    11|-0.43821824|
|     67|    11|-0.59333915|
|     72|    11|-0.80404276|
+-------+------+-----------+

